## Apress - Industrialized Machine Learning Examples

Andreas Francois Vermeulen
2019

### This is an example add-on to a book and needs to be accepted as part of that copyright.

## Chapter-011-20-Telecomms-01

In [1]:
#!pip install simpy

## Part A - Load Libraries

In [2]:
import random
import simpy
import os

RANDOM_SEED = 1968
SIM_TIME = 500

## Part B - Configure Telecommunication Broadcast Pipe

In [3]:
class Telecom_BroadCast_Pipe(object):
    """
    A broadcast pipe that allows one process to send messages to many other process.
    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Broadcast a *value* to all receivers."""
        if not self.pipes:
            raise RuntimeError('There are no output pipes.')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self):
        """Get a new output connection for this broadcast pipe.
        The return value is a :class:`~simpy.resources.store.Store`.
        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe

## Part C - Configure Telecommunication Message Generator

In [4]:
def Telecom_Message_Generator(name, env, out_pipe):
    """A process that generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(6, 10))

        # messages are time stamped to later check if the consumer was
        # late getting them.  Note, using event.triggered to do this may
        # result in failure due to FIFO nature of simulation yields.
        # (i.e. if at the same env.now, message_generator puts a message
        # in the pipe first and then message_consumer gets from pipe,
        # the event.triggered will be True in the other order it will be
        # False
        msg = (env.now, '%s sends ML command at %d' % (name, env.now))
        MsgLine=msg
        Log_Message(MsgLine)
        out_pipe.put(msg)

## Part D - Configure Telecommunication End-Consumer

In [5]:
def Telecom_Message_Consumer(name, env, in_pipe):
    """A process that consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # if message was already put into pipe, then
            # message_consumer was late getting to it. 
            MsgLine = ('FAILED Message: at time %d: %s received the command: %s' % (env.now, name, msg[1]))
            Log_Message(MsgLine)

        else:
            # message_consumer is synchronized with message_generator
            MsgLine = ('SUCCESS Message: at time %d: %s received the command: %s.' % (env.now, name, msg[1]))
            Log_Message(MsgLine)

        # Process does some other work, this may result in missing messages
        yield env.timeout(random.randint(4, 8))


## Part E - Setup and start the Telecom Simulation

In [6]:
print('Process Telecommunication Setup and Started')
random.seed(RANDOM_SEED)
env = simpy.Environment()

Process Telecommunication Setup and Started


In [7]:
def Log_Message(MsgLine):
    print(MsgLine)
    MsgLineWrite=str(MsgLine) + '\n'
    f.writelines(MsgLineWrite)

In [8]:
imagepath = os.path.join(*[os.path.dirname(os.path.dirname(os.getcwd())),'Results','Chapter 11'])
print(imagepath)

if not os.path.exists(imagepath):
    os.makedirs(imagepath)

C:\Users\sirve\OneDrive\My Documents\My Book\Results\Chapter 11


## Part F - Wired Communication Simulation

In [9]:
ReportName='Chapter-011-20-Telecomms-01-01.txt'
ReportFullName = os.path.join(*[os.path.dirname(os.path.dirname(os.getcwd())),'Results','Chapter 11',ReportName])
print(ReportFullName)

C:\Users\sirve\OneDrive\My Documents\My Book\Results\Chapter 11\Chapter-011-20-Telecomms-01-01.txt


In [10]:
f = open(ReportFullName,"w") 

In [11]:
pipe = simpy.Store(env)
env.process(Telecom_Message_Generator('Transmit A', env, pipe))
env.process(Telecom_Message_Consumer('Receive  Beta', env, pipe))

MsgLine = ('\nWired Telecommunication - Start\n')
Log_Message(MsgLine)

env.run(until=SIM_TIME)

MsgLine = ('\nWired Telecommunication - Stop\n')
Log_Message(MsgLine)


Wired Telecommunication - Start

(7, 'Transmit A sends ML command at 7')
SUCCESS Message: at time 7: Receive  Beta received the command: Transmit A sends ML command at 7.
(13, 'Transmit A sends ML command at 13')
FAILED Message: at time 15: Receive  Beta received the command: Transmit A sends ML command at 13
(22, 'Transmit A sends ML command at 22')
SUCCESS Message: at time 22: Receive  Beta received the command: Transmit A sends ML command at 22.
(29, 'Transmit A sends ML command at 29')
SUCCESS Message: at time 29: Receive  Beta received the command: Transmit A sends ML command at 29.
(37, 'Transmit A sends ML command at 37')
SUCCESS Message: at time 37: Receive  Beta received the command: Transmit A sends ML command at 37.
(47, 'Transmit A sends ML command at 47')
SUCCESS Message: at time 47: Receive  Beta received the command: Transmit A sends ML command at 47.
(54, 'Transmit A sends ML command at 54')
SUCCESS Message: at time 54: Receive  Beta received the command: Transmit A se

In [12]:
f.close()

## Part G - Central Node Wireless Communication Simulation

In [13]:
ReportName='Chapter-011-20-Telecomms-01-02.txt'
ReportFullName = os.path.join(*[os.path.dirname(os.path.dirname(os.getcwd())),'Results','Chapter 11',ReportName])
print(ReportFullName)

C:\Users\sirve\OneDrive\My Documents\My Book\Results\Chapter 11\Chapter-011-20-Telecomms-01-02.txt


In [14]:
f = open(ReportFullName,"w") 

In [15]:
env = simpy.Environment()
bc_pipe = Telecom_BroadCast_Pipe(env)

env.process(Telecom_Message_Generator('Transmit Alpha', env, bc_pipe))
env.process(Telecom_Message_Consumer('Receive Beta', env, bc_pipe.get_output_conn()))
env.process(Telecom_Message_Consumer('Receive Charlie', env, bc_pipe.get_output_conn()))

MsgLine = ('\nCentral Wireless Telecommunication - Start\n')
Log_Message(MsgLine)

env.run(until=SIM_TIME)

MsgLine = ('\nCentral Wireless Telecommunication - Stop\n')
Log_Message(MsgLine)


Central Wireless Telecommunication - Start

(6, 'Transmit Alpha sends ML command at 6')
SUCCESS Message: at time 6: Receive Beta received the command: Transmit Alpha sends ML command at 6.
SUCCESS Message: at time 6: Receive Charlie received the command: Transmit Alpha sends ML command at 6.
(16, 'Transmit Alpha sends ML command at 16')
SUCCESS Message: at time 16: Receive Beta received the command: Transmit Alpha sends ML command at 16.
SUCCESS Message: at time 16: Receive Charlie received the command: Transmit Alpha sends ML command at 16.
(24, 'Transmit Alpha sends ML command at 24')
SUCCESS Message: at time 24: Receive Beta received the command: Transmit Alpha sends ML command at 24.
SUCCESS Message: at time 24: Receive Charlie received the command: Transmit Alpha sends ML command at 24.
(31, 'Transmit Alpha sends ML command at 31')
SUCCESS Message: at time 31: Receive Charlie received the command: Transmit Alpha sends ML command at 31.
FAILED Message: at time 32: Receive Beta rec

SUCCESS Message: at time 358: Receive Beta received the command: Transmit Alpha sends ML command at 358.
SUCCESS Message: at time 358: Receive Charlie received the command: Transmit Alpha sends ML command at 358.
(365, 'Transmit Alpha sends ML command at 365')
SUCCESS Message: at time 365: Receive Beta received the command: Transmit Alpha sends ML command at 365.
SUCCESS Message: at time 365: Receive Charlie received the command: Transmit Alpha sends ML command at 365.
(375, 'Transmit Alpha sends ML command at 375')
SUCCESS Message: at time 375: Receive Beta received the command: Transmit Alpha sends ML command at 375.
SUCCESS Message: at time 375: Receive Charlie received the command: Transmit Alpha sends ML command at 375.
(382, 'Transmit Alpha sends ML command at 382')
SUCCESS Message: at time 382: Receive Beta received the command: Transmit Alpha sends ML command at 382.
SUCCESS Message: at time 382: Receive Charlie received the command: Transmit Alpha sends ML command at 382.
(392

In [16]:
f.close()

## Part H - Single-Channel-Wireless Communication Simulation

In [17]:
ReportName='Chapter-011-20-Telecomms-01-03.txt'
ReportFullName = os.path.join(*[os.path.dirname(os.path.dirname(os.getcwd())),'Results','Chapter 11',ReportName])
print(ReportFullName)

C:\Users\sirve\OneDrive\My Documents\My Book\Results\Chapter 11\Chapter-011-20-Telecomms-01-03.txt


In [18]:
f = open(ReportFullName,"w") 

In [19]:
env = simpy.Environment()
bc_pipe = Telecom_BroadCast_Pipe(env)

env.process(Telecom_Message_Generator('Transmit Alpha', env, bc_pipe))
env.process(Telecom_Message_Consumer('Receive Alpha', env, bc_pipe.get_output_conn()))

env.process(Telecom_Message_Generator('Transmit Beta', env, bc_pipe))
env.process(Telecom_Message_Consumer('Receive Beta', env, bc_pipe.get_output_conn()))

env.process(Telecom_Message_Generator('Transmit Charlie', env, bc_pipe))
env.process(Telecom_Message_Consumer('Receive Charlie', env, bc_pipe.get_output_conn()))

MsgLine = ('\nSingle-Channel Wireless Telecommunication - Start\n')
Log_Message(MsgLine)

env.run(until=SIM_TIME)

MsgLine = ('\nSingle-Channel Wireless Telecommunication - Stop\n')
Log_Message(MsgLine)


Single-Channel Wireless Telecommunication - Start

(9, 'Transmit Alpha sends ML command at 9')
(9, 'Transmit Beta sends ML command at 9')
SUCCESS Message: at time 9: Receive Alpha received the command: Transmit Alpha sends ML command at 9.
SUCCESS Message: at time 9: Receive Beta received the command: Transmit Alpha sends ML command at 9.
SUCCESS Message: at time 9: Receive Charlie received the command: Transmit Alpha sends ML command at 9.
(10, 'Transmit Charlie sends ML command at 10')
(15, 'Transmit Beta sends ML command at 15')
FAILED Message: at time 15: Receive Charlie received the command: Transmit Beta sends ML command at 9
(17, 'Transmit Charlie sends ML command at 17')
FAILED Message: at time 17: Receive Alpha received the command: Transmit Beta sends ML command at 9
FAILED Message: at time 17: Receive Beta received the command: Transmit Beta sends ML command at 9
(19, 'Transmit Alpha sends ML command at 19')
FAILED Message: at time 20: Receive Charlie received the command: 

FAILED Message: at time 466: Receive Charlie received the command: Transmit Alpha sends ML command at 216
FAILED Message: at time 470: Receive Beta received the command: Transmit Charlie sends ML command at 206
FAILED Message: at time 470: Receive Alpha received the command: Transmit Charlie sends ML command at 214
(471, 'Transmit Charlie sends ML command at 471')
(471, 'Transmit Beta sends ML command at 471')
(472, 'Transmit Alpha sends ML command at 472')
FAILED Message: at time 474: Receive Charlie received the command: Transmit Beta sends ML command at 219
FAILED Message: at time 476: Receive Beta received the command: Transmit Alpha sends ML command at 209
(477, 'Transmit Charlie sends ML command at 477')
FAILED Message: at time 477: Receive Alpha received the command: Transmit Alpha sends ML command at 216
(480, 'Transmit Beta sends ML command at 480')
FAILED Message: at time 481: Receive Charlie received the command: Transmit Charlie sends ML command at 222
(482, 'Transmit Alpha

## Part J - Tri-Mesh-Wireless Communication Simulation

In [20]:
ReportName='Chapter-011-20-Telecomms-01-04.txt'
ReportFullName = os.path.join(*[os.path.dirname(os.path.dirname(os.getcwd())),'Results','Chapter 11',ReportName])
print(ReportFullName)

C:\Users\sirve\OneDrive\My Documents\My Book\Results\Chapter 11\Chapter-011-20-Telecomms-01-04.txt


In [21]:
f = open(ReportFullName,"w") 

In [22]:
env = simpy.Environment()
bc_pipe_A = Telecom_BroadCast_Pipe(env)
bc_pipe_B = Telecom_BroadCast_Pipe(env)
bc_pipe_C = Telecom_BroadCast_Pipe(env)

env.process(Telecom_Message_Generator('Transmit Alpha', env, bc_pipe_A))
env.process(Telecom_Message_Consumer('Receive Alpha-Beta', env, bc_pipe_B.get_output_conn()))
env.process(Telecom_Message_Consumer('Receive Alpha-Charlie', env, bc_pipe_C.get_output_conn()))

env.process(Telecom_Message_Generator('Transmit Beta', env, bc_pipe_B))
env.process(Telecom_Message_Consumer('Receive Beta-Alpha', env, bc_pipe_A.get_output_conn()))
env.process(Telecom_Message_Consumer('Receive Beta-Charlie', env, bc_pipe_C.get_output_conn()))

env.process(Telecom_Message_Generator('Transmit Charlie', env, bc_pipe_C))
env.process(Telecom_Message_Consumer('Receive Charlie-Alpha', env, bc_pipe_A.get_output_conn()))
env.process(Telecom_Message_Consumer('Receive Charlie-Beta', env, bc_pipe_B.get_output_conn()))

MsgLine = ('\nTri-Mesh Wireless Telecommunication - Start\n')
Log_Message(MsgLine)

env.run(until=SIM_TIME)

MsgLine = ('\nTri-Mesh Wireless Telecommunication - Stop\n')
Log_Message(MsgLine)


Tri-Mesh Wireless Telecommunication - Start

(7, 'Transmit Alpha sends ML command at 7')
(7, 'Transmit Charlie sends ML command at 7')
SUCCESS Message: at time 7: Receive Beta-Alpha received the command: Transmit Alpha sends ML command at 7.
SUCCESS Message: at time 7: Receive Charlie-Alpha received the command: Transmit Alpha sends ML command at 7.
SUCCESS Message: at time 7: Receive Alpha-Charlie received the command: Transmit Charlie sends ML command at 7.
SUCCESS Message: at time 7: Receive Beta-Charlie received the command: Transmit Charlie sends ML command at 7.
(10, 'Transmit Beta sends ML command at 10')
SUCCESS Message: at time 10: Receive Alpha-Beta received the command: Transmit Beta sends ML command at 10.
SUCCESS Message: at time 10: Receive Charlie-Beta received the command: Transmit Beta sends ML command at 10.
(16, 'Transmit Alpha sends ML command at 16')
SUCCESS Message: at time 16: Receive Beta-Alpha received the command: Transmit Alpha sends ML command at 16.
SUCCES

SUCCESS Message: at time 235: Receive Alpha-Charlie received the command: Transmit Charlie sends ML command at 235.
SUCCESS Message: at time 235: Receive Beta-Charlie received the command: Transmit Charlie sends ML command at 235.
(236, 'Transmit Alpha sends ML command at 236')
SUCCESS Message: at time 236: Receive Beta-Alpha received the command: Transmit Alpha sends ML command at 236.
SUCCESS Message: at time 236: Receive Charlie-Alpha received the command: Transmit Alpha sends ML command at 236.
(240, 'Transmit Beta sends ML command at 240')
SUCCESS Message: at time 240: Receive Alpha-Beta received the command: Transmit Beta sends ML command at 240.
SUCCESS Message: at time 240: Receive Charlie-Beta received the command: Transmit Beta sends ML command at 240.
(242, 'Transmit Charlie sends ML command at 242')
SUCCESS Message: at time 242: Receive Beta-Charlie received the command: Transmit Charlie sends ML command at 242.
SUCCESS Message: at time 242: Receive Alpha-Charlie received t

In [23]:
f.close()

## Done

In [24]:
import datetime
now = datetime.datetime.now()
print('Done!',str(now))

Done! 2019-04-29 21:00:00.020544
